In [0]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache, reduce
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D

In [0]:
# this code ran in colab and used files from my google-drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# load files into arrays
fashion_train_array = np.load('/content/drive/My Drive/zalando_all/fashion_train.npy') # 10000,785
all_input = np.load('/content/drive/My Drive/zalando_all/fashion_train.npy') # 10000,785
fashion_test_array = np.load('/content/drive/My Drive/zalando_all/fashion_test.npy') # 5000, 785
dilated_train_array = np.load('/content/drive/My Drive/zalando_all/dilated_input.npy') # 10000, 784
lda3_projector = np.load('/content/drive/My Drive/zalando_all/sorted_eigvecs_3.npy')
lda4_projector = np.load('/content/drive/My Drive/zalando_all/sorted_eigvecs_4.npy')

In [0]:
# take labels and features from train and test
fashion_train_features = fashion_train_array[:, :-1]
fashion_train_labels = fashion_train_array[:, -1]
fashion_test_features = fashion_test_array[:, :-1]
fashion_test_labels = fashion_test_array[:, -1]

In [0]:
# shuffle and normalization
np.random.shuffle(all_input)
fashion_train_features = fashion_train_features / np.max(fashion_train_features)
fashion_test_features = fashion_test_features / np.max(fashion_train_features)

In [0]:
# label extraction
labels = all_input[:, -1]
current_train = all_input[:, :-1]

# projecting normalized train and test set with lda
d3_projected = np.dot(current_train, lda3_projector)
d4_projected = np.dot(current_train, lda4_projector)

# smae for test set
d3_projected_test = np.dot(fashion_test_features, lda3_projector)
d4_projected_test = np.dot(fashion_test_features, lda4_projector)

In [0]:
# create different label inputs for classifier
labels_0, labels_1, labels_2, labels_3, labels_4 = labels.astype(np.int8), labels.astype(np.int8), labels.astype(np.int8), labels.astype(np.int8), labels.astype(np.int8)
labels_all = [labels_0, labels_1, labels_2, labels_3, labels_4]
for idx,l in enumerate(labels_all):
  l[labels == idx] = 1
  l[labels != idx] = -1

In [0]:
# just for check
np.unique(labels_3, return_counts = True)

(array([-1,  1], dtype=int8), array([7995, 2005]))

In [0]:
# cross_validation datasets
d3_train_x_1 = d3_projected[:8000, :]
d3_train_x_2 = d3_projected[2000:, :]
d3_train_x_3 = np.concatenate((d3_projected[:2000, :], d3_projected[4000:, :]), axis = 0)
d3_train_x_4 = np.concatenate((d3_projected[:4000, :], d3_projected[6000:, :]), axis = 0)
d3_train_x_5 = np.concatenate((d3_projected[:6000, :], d3_projected[8000:, :]), axis = 0)

d3_test_x_1 = d3_projected[8000:]
d3_test_x_2 = d3_projected[:2000]
d3_test_x_3 = d3_projected[2000:4000]
d3_test_x_4 = d3_projected[4000:6000]
d3_test_x_5 = d3_projected[6000:8000]

d4_train_x_1 = d4_projected[:8000, :]
d4_train_x_2 = d4_projected[2000:, :]
d4_train_x_3 = np.concatenate((d4_projected[:2000, :], d4_projected[4000:, :]), axis = 0)
d4_train_x_4 = np.concatenate((d4_projected[:4000, :], d4_projected[6000:, :]), axis = 0)
d4_train_x_5 = np.concatenate((d4_projected[:6000, :], d4_projected[8000:, :]), axis = 0)

d4_test_x_1 = d4_projected[8000:]
d4_test_x_2 = d4_projected[:2000]
d4_test_x_3 = d4_projected[2000:4000]
d4_test_x_4 = d4_projected[4000:6000]
d4_test_x_5 = d4_projected[6000:8000]

In [0]:
# method built mostly from hw7 notebook
def run_classifer(train_x, train_y, test_x, test_y, batch_size, train_steps): # session started
  # start session
  sess = tf.Session()
  # give according lda-d data you want to feed
  dim = train_x.shape[1]

  # init placeholders
  x = tf.placeholder(shape = [None, dim], dtype = tf.float32)
  r = tf.placeholder(shape = [None, 1], dtype = tf.float32)
  prediction_grid = tf.placeholder(shape = [None, dim], dtype = tf.float32)
  prediction_r = tf.placeholder(shape = [None, 1], dtype = tf.float32)

  # create variable fro svm
  alpha = tf.Variable(tf.random_normal(shape = [1, batch_size]))

  #declare kernel gaussian kernel
  gamma = tf.constant(2.0)
  dist = tf.reduce_sum(tf.square(x), 1)
  sq_dist = tf.subtract(tf.add(dist, dist), tf.multiply(2.0, tf.matmul(x, tf.transpose(x))))
  kernel = tf.exp(tf.multiply(-gamma, tf.abs(sq_dist)))

  # declare linear kernel
  #kernel = tf.matmul(x, tf.transpose(x))
  #kernel = tf.tensordot(x, tf.transpose(x), 1)

  # prediction kernel gaussian
  tr = tf.reshape(tf.reduce_sum(tf.square(x), 1), [-1,1])
  te = tf.reduce_sum(tf.square(prediction_grid), 1)
  pred_sq_dist = tf.add(tf.subtract(tr, tf.multiply(2.0, tf.matmul(x, tf.transpose(prediction_grid)))), te)
  pred_kernel = tf.exp(tf.multiply(-gamma, tf.abs(pred_sq_dist)))

  # linear prediction kernel
  #pred_kernel = tf.matmul(x, tf.transpose(prediction_grid))
  #pred_kernel = tf.tensordot(x, tf.transpose(prediction_grid), 1)

  # compute loss function
  alpha_sum = tf.reduce_sum(alpha)

  r_matrix = tf.matmul(r, tf.transpose(r))
  alpha_prod = tf.matmul(tf.transpose(alpha), alpha)
  double_sum = tf.reduce_sum(tf.multiply(kernel, tf.multiply(alpha_prod, r_matrix)))

  loss = tf.negative(tf.subtract(alpha_sum, double_sum))

  # predict
  pred_output = tf.matmul(tf.multiply(tf.transpose(r), alpha), pred_kernel)
  norm_pred_output = pred_output - tf.reduce_mean(pred_output)  
  #pred = tf.sign(pred_output - tf.reduce_mean(pred_output))
  pred = tf.sign(pred_output)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(pred), tf.squeeze(prediction_r)), tf.float32))

  # declare optimizer
  my_opt = tf.train.GradientDescentOptimizer(0.0005)
  train_step = my_opt.minimize(loss)

  init = tf.global_variables_initializer()
  sess.run(init)

  # training
  temp_losses = []
  batch_accs = []
  np.random.seed(265)
  alphas = []
  r_s = []
  pred_outputs = []
  for i in range(train_steps):
    batch_idxs = np.random.choice(8000, size = batch_size)
    batch_x = train_x[batch_idxs]
    batch_r = np.transpose([train_y[batch_idxs]])
    sess.run(train_step, feed_dict= {x:batch_x, r:batch_r})

    temp_loss = sess.run(loss, feed_dict={x:batch_x, r:batch_r})
    temp_losses.append(temp_loss)

    current_pred_out = sess.run(pred_output, feed_dict = {x: batch_x, r: batch_r, prediction_grid: test_x, prediction_r : np.transpose([test_y])})
    pred_outputs.append(current_pred_out)

    current_alpha = sess.run(alpha, feed_dict = {x: batch_x, r: batch_r, prediction_grid: test_x, prediction_r : np.transpose([test_y])})
    alphas.append(current_alpha)

    current_r = sess.run(r, feed_dict = {x: batch_x, r: batch_r, prediction_grid: test_x, prediction_r : np.transpose([test_y])})
    r_s.append(current_r)

    batch_acc = sess.run(accuracy, feed_dict = {x: batch_x, r: batch_r, prediction_grid: test_x, prediction_r : np.transpose([test_y])})
    batch_accs.append(batch_acc)

    if (i+1)%(int(train_steps/6)) == 0:
      print("Step #{}".format(i+1))
      print("Loss = ", temp_loss)
      #print(f"x: {sess.run(x, feed_dict={x:batch_x, r:batch_r}).shape}")
      #print(f"r: {sess.run(r, feed_dict={x:batch_x, r:batch_r}).shape}")
      #print(f"alpha: {sess.run(alpha, feed_dict={x:batch_x, r:batch_r})}")
      print(sess.run(accuracy, feed_dict = {x: batch_x, r: batch_r, prediction_grid: test_x, prediction_r : np.transpose([test_y])}))
      print(sess.run(pred_kernel, feed_dict = {x: batch_x, r: batch_r, prediction_grid: test_x, prediction_r : np.transpose([test_y])}).shape)
    
  sess.close()
  return batch_accs[-1], pred_outputs[-1]

In [0]:
# running the training
pred_outs = []
accs = []
for current_labels in labels_all:
  current_accs, current_pred_outs = run_classifer(train_x = d4_train_x_1, train_y = current_labels[:8000], test_x = d4_test_x_1, test_y = current_labels[8000:], batch_size = 500, train_steps = 3000)
  pred_outs.append(current_pred_outs)
  accs.append(current_accs)

Step #500
Loss =  139.30219
0.6325
(500, 2000)
Step #1000
Loss =  9.597321
0.7675
(500, 2000)
Step #1500
Loss =  -27.07608
0.8815
(500, 2000)
Step #2000
Loss =  -39.673294
0.92
(500, 2000)
Step #2500
Loss =  -55.318123
0.9285
(500, 2000)
Step #3000
Loss =  -49.283123
0.924
(500, 2000)
Step #500
Loss =  144.4025
0.667
(500, 2000)
Step #1000
Loss =  28.830574
0.7645
(500, 2000)
Step #1500
Loss =  -18.231041
0.872
(500, 2000)
Step #2000
Loss =  -28.252453
0.961
(500, 2000)
Step #2500
Loss =  -38.512108
0.988
(500, 2000)
Step #3000
Loss =  -40.60344
0.9875
(500, 2000)
Step #500
Loss =  138.4274
0.613
(500, 2000)
Step #1000
Loss =  22.154938
0.763
(500, 2000)
Step #1500
Loss =  -18.654037
0.8775
(500, 2000)
Step #2000
Loss =  -37.196045
0.9365
(500, 2000)
Step #2500
Loss =  -45.73643
0.949
(500, 2000)
Step #3000
Loss =  -44.41781
0.9465
(500, 2000)
Step #500
Loss =  139.88055
0.639
(500, 2000)
Step #1000
Loss =  14.980392
0.801
(500, 2000)
Step #1500
Loss =  -27.887558
0.871
(500, 2000)
Ste

In [0]:
# calculating accuracy
accuracy = 0
confusion_matrix = np.zeros((5,5))
max_arrays = np.zeros((2000,5), dtype = np.float64)
predictions = np.zeros(2000, dtype = np.int8)
for idx, label in enumerate(labels[8000:]):
  max_array = np.array([pred_outs[0][0][idx], pred_outs[1][0][idx], pred_outs[2][0][idx], pred_outs[3][0][idx], pred_outs[4][0][idx]])
  max_arrays[idx, :] = max_array
  predictions[idx] = np.argmax(max_array)
  if label == np.argmax(max_array):
    accuracy += 1
  confusion_matrix[label, np.argmax(max_array)] += 1


In [0]:
confusion_matrix

array([[337.,   0.,  10.,  17.,  31.],
       [  0., 385.,   1.,   8.,   2.],
       [  9.,   3., 353.,   7.,  27.],
       [ 25.,   9.,   7., 346.,  16.],
       [ 64.,   0.,  44.,  15., 284.]])

In [0]:
# not acc
accuracy / 2000

0.8525

In [0]:
predictions.shape

(2000,)

In [0]:
import pandas as pd
my_df = pd.DataFrame({'true': list(labels[8000:]), 'predicted': predictions})

In [0]:
# save labels
from google.colab import files
my_df.to_csv('d4_cv_1.csv', header = True, index = False, sep = ',')
files.download('d4_cv_1.csv')

In [0]:
max_df = pd.DataFrame(max_arrays, columns = list('01234'))

In [0]:
# all projections
max_df

,0,1,2,3,4
0,-0.002150,0.000823,-0.001684,-0.001253,-0.001750
1,0.208151,-0.100838,-0.181143,-0.116698,-0.212389
2,-0.205672,-0.174433,-0.219351,0.183180,-0.223888
3,0.000898,-0.004479,-0.005336,-0.003891,-0.002175
4,-0.269537,-0.199138,-0.257518,0.290814,-0.340472
...,...,...,...,...,...
4995,-0.020336,-0.011938,-0.019064,0.017257,-0.017996
4996,-1.100574,0.914975,-1.065973,-1.073578,-1.257206
4997,-0.000246,-0.000117,0.000258,-0.000250,-0.000267
4998,-0.010151,-0.174000,-0.202427,-0.178860,-0.005832


In [0]:
max_df.to_csv('ld4_max.csv', header = True, index = False, sep = ',')
files.download('ld4_max.csv')